# KDD Cup 2009

#### Thullio Debortoli Moreira Zanetti

## Descrição do problema

Em 2009, o KDD Cup forneceu um conjunto de dados de gestão de relacionamento com o cliente. Esse tipo de dados é comumente conhecido como customer relationship management (CRM).  Foi disponibilizado 230 fatos sobre 50000 contas de cartão de crédito. Com esses atributos o objetivo é predizer o cancelamento da conta (churn), a tendência de usar novos produtos e serviços (appetency) e a propensão para comprar upgrades ou adicionais quando apresentados (up-selling).

O objetivo deste exercício é a criação de três modelos de Inteligência Artificial para realizar a predição de cada uma das métricas citadas (churn, appetency e op-selling), baseado nos dados de uma empresa de telecomunicação Francesa, a Orange, e disponibilizado no KDD Cup de 2009. O exercício foi implementado com a base de dados menor (50000 exemplos com 230 características cada um) por motivos de processamento.

Os modelos de Inteligência Artificial utilizados neste exercício serão modelos de classificação - tipo de aprendizado supervisionado (aprendizado de máquina no qual se tem informações prévias de entrada e saída esperadas, que são realizadas a partir medições ou observações, e que é o caso da base de dados utilizada) no qual pretende-se classificar cada conjunto de entradas em classes diferentes (no caso da base de dados da Orange, os clientes são classificados como Verdadeiro ou Falso para cada uma das métricas) - e serão avaliados pela média do AUC, que mede a capacidade discriminatória do modelo, isto é, a capacidade do modelo separar as amostras entre as classes corretamente.

O exercício foi dividido em 3 etapas, que são apresentadas a seguir.

## 1) EDA

O EDA, ou exploratory data analysis (em tradução livre, análise exploratória dos dados), busca compreender a estrutura e o comportamento, assim como realizar modificações necessárias, pois os dados obtidos em situações reais, geralmente, não são limpos nem homogêneos e precisam ser tratados previamente.

O primeiro passo do para tratamento dos dados utilizados, após a leitura da base de dados foi a exclusão de fatos (ou características) que não estão contidos em mais de 50% dos clientes, tendo em vista que não são informações que contém exemplos suficientes para serem abstraídas pelos modelos de Inteligência Artificial.

Após a realização desta etapa, sobram 69 características.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Importing training data
df = pd.read_csv('orange_small_train.data/orange_small_train.data', sep = '\t')
df_test = pd.read_csv('orange_small_test.data/orange_small_test.data', sep = '\t')
tt_appetency = pd.read_csv('orange_small_train_appetency.labels.txt', sep = '\t', header=None, names = ["appetency"])
tt_churn = pd.read_csv('orange_small_train_churn.labels', sep = '\t', header=None, names = ["churn"])
tt_upselling = pd.read_csv('orange_small_train_upselling.labels.txt', sep = '\t', header=None, names = ["upselling"])

#Transforming outputs to boolean
tt_upselling["upselling"] = tt_upselling["upselling"].apply(lambda x : 1 if x == 1 else 0)
tt_churn["churn"] = tt_churn["churn"].apply(lambda x : 1 if x == 1 else 0)
tt_appetency["appetency"] = tt_appetency["appetency"].apply(lambda x : 1 if x == 1 else 0)

#dropping columns with more than 50% of NaN
columns_to_keep = df.dropna(axis=1, thresh=int(df.shape[0]*0.5)).columns
df = df[columns_to_keep]
df_test = df_test[columns_to_keep]
df.shape

(50000, 69)

O segundo passo para o tratamento dos valores não conhecidos é completá-los de acordo com as informações dos outros exemplos. Para isto, as características que são números serão completadas com a mediana dos outros valores, visando retirar a influência dos outliers (valores muito grandes ou muito pequenos) presentes na amostra, e os fatos do tipo texto serão completados com a informação "OUTROS" (considerando que são grupos não classificados previamente).

In [3]:
df.update(df.select_dtypes(include='O').fillna("OUTROS"))
df_numbers_median = df.select_dtypes(exclude='O').median()
df.update(df.select_dtypes(exclude='O').fillna(df_numbers_median))

df_test.update(df_test.select_dtypes(include='O').fillna("OUTROS"))
df_numbers_median = df_test.select_dtypes(exclude='O').median()
df_test.update(df_test.select_dtypes(exclude='O').fillna(df_numbers_median))

O próximo passo para tratamento da base de dados é o tratamento das características que são do tipo texto. Atributos do tipo texto não fornecem informação para modelos de Inteligência Artificial, que são modelos matemáticos e tem que ser alimentados com números. 

Primeiramente será realizado um novo filtro nas características, verificando se os textos agrupam os dados de forma a gerarem informação relevantes para o modelo, ou se são informações específicas para cada cliente e que não trará informações para o modelo. Definiu-se que, levando em consideração que são 100000 clientes(juntando os datasets de treinamento e teste), os textos não podem conter mais de 0,05% (50) de valores únicos, pois assim, teoricamente, tem-se cerca de 2000 clientes por grupo e trará informação necessária para o modelo, sem que tenha um grande aumento no número de caracteríticas com a utilização da técnica explicada a seguir.

OBS: Os datasets de treinamento e teste foram concatenados neste passo pois percebeu-se que existiam características do tipo texto que tinham valores no dataset de teste, mas nao do de treinamento, fazendo com que no final tivessem número de características diferente.

A segunda parte de transformação destas informações de texto são passá-las para número. Para isto será utilizada uma técnica de categorização dos dados, que converte cada uma das categorias em uma variável do tipo "dummy", que consiste em para cada categoria criar uma nova característica que será binária (1 ou 0), e cada um dos clientes terá apenas uma das colunas criadas como 1, sendo a correspondente à sua categoria.

In [4]:
df_aux = pd.concat([df, df_test]).reset_index()

#Drop columns with more than 50 unique values
number_unique_values = df_aux.select_dtypes(include='O').nunique()
df_aux.drop(number_unique_values[number_unique_values > 50].index, axis=1, inplace = True)

#Convert text columns into dummies
text_columns = df_aux.select_dtypes(include='O')
new_features = pd.get_dummies(text_columns, drop_first=True)
df_aux = pd.concat([df_aux, new_features], axis=1)
df_aux.drop(text_columns.columns, axis=1, inplace = True)

df = df_aux.loc[:49999].reset_index()
df_test = df_aux.loc[50000:].reset_index()

O último passo do EDA é realizar a normalização dos dados. Isto é feito por causa da importância dos dados estarem em uma mesma escala para os algoritmos de machine learning. Alguns ganhos com a normalização:

-> Como se tratam de modelos matemáticos, escalas uniformes refletem em algoritmos que realizam suas funções de convergência para otimização dos modelos de forma mais rápida

-> Influência mais "justa"ou uniforme de todas as características no modelo

Para o exercício foi escolhido a normalização Z-score, que tem como fórmula:

\begin{equation*}
y = \frac{x - \mu}{\sigma^2}
\end{equation*}

Sendo $\mu$ a média e $\sigma^2$ a variância de cada uma das características.

A normalização escolhida busca dispor os dados de forma que apresentem média igual a 0 e normalizar a variância de cada característica como 1.

In [5]:
mean = df.mean()
var = pow(df.std(), 2)

df = (df-mean)/(var + 0.00001)
df_test = (df_test-mean)/(var + 0.00001)

Antes de realizar a escolha dos modelos, primeiramente deve-se analisar algumas características do dataset com as características finais. Para isto, analisou-se a correlação entre os dados e a saída. A correlação é uma informação importante para saber se as características estao correlacionadas (diretamente ou inversamente proporcional) às saídas que estão sendo predizidas. Para nosso problema a correlação dos dados com a saída não apresentam valores significativos, logo modelos lineares (como regressão linear ou logística) não são sugeridos a serem utilizados, pois não dariam resultados significativos.

In [40]:
df_aux = pd.concat([df, tt_upselling, tt_churn, tt_appetency], axis=1)
correlation_abs = np.absolute(df_aux.corr()[['upselling', 'churn', 'appetency']])
print(correlation_abs.nlargest(5, 'upselling')['upselling'])
print(correlation_abs.nlargest(5, 'churn')['churn'])
print(correlation_abs.nlargest(5, 'appetency')['appetency'])

upselling      1.000000
Var126         0.138897
Var211_Mtgm    0.131556
Var28          0.096780
Var153         0.080460
Name: upselling, dtype: float64
churn             1.000000
upselling         0.079377
Var205_sJzTlal    0.071669
Var73             0.067534
Var7              0.066316
Name: churn, dtype: float64
appetency      1.000000
Var126         0.113907
Var218_cJvF    0.096413
Var218_UYBR    0.093614
Var211_Mtgm    0.062471
Name: appetency, dtype: float64


Também verificou-se que a base de dados tem uma característica conhecida como Classes desbalanceadas. O desbalanceamento de classes corresponde quando uma classe é representada por muitos exemplos, enquanto a outra classe é representado apenas por alguns exemplos. No caso deste trabalho as classes positivas (representadas como 1) tem muito menos exemplos do que a classe negativa.

Para tratar este obstáculo existe algumas técnicas, as técnicas escolhidas são listadas abaixo.

O primeiro tratamento feito com os dados é uma técnica que busca de igualar o número de amostras de cada classe. O algoritmo SMOTE é uma técnica de pré-processamento de dados que realiza o oversampling da classe minoritária, isto é, aumenta o número de amostras da classe que têm o menor número de exemplos. O SMOTE tem o objetivo de obter impacto positivo na fronteira de decisão após os treinamentos dos dados, e para isso gera um aumento no número de amostras da classe minoritária por meio da criação de amostras sintéticas. O algoritmo segue os seguintes passos:

Primeiramente são selecionados os k vizinhos mais próximos para cada uma das amostras minoritárias e são selecionados n destes vizinhos para criação das amostras sintéticas. O valor de n é definido a partir do quanto quer se aumentar a amostra, por exemplo se deseja-se dobrar o numero de amostras, então apenas dois dos k vizinhos mais próximos de cada amostra original são selecionados para a criação das amostras sintéticas. Então, para cada um dos n vizinhos de cada uma das amostras são seguidos os seguintes passos:

1. calcula-se a diferença entre a amostra original e seu vizinho
2. multiplica-se este valor da diferença por um número aleatório entre 0 e 1
3. o valor calculado é adicionado a amostra original, obtendo-se a amostra sintética

A figura abaixo mostra, graficamente, o funcionamento do algoritmo SMOTE.

<img src="https://cdn-images-1.medium.com/max/1200/1*6UFpLFl59O9e3e38ffTXJQ.png" width=400px>

Para não ocorrer efeito de perder características importantes, devido à geração de amostras aleatórias, ao final do SMOTE a classe majoritária ainda será 4 vezes maior que a classe minoritária e serão utilizadas outras técnicas para tratamento do desbalanceamento restante.

Devido à grande diferença entre os números das classes (classe maioritária até 55 vezes maior que a classe minoritária), foi de utilizar uma função de "loss", que é utilizada como função de otimização, isto é, o modelo tentará minimizar ou maximizar esta função. Para este trabalho será utilizado um cálculo aproximado do AUC (como é a métrica de avaliação) proposto por Yan, L., Dodier, R., Mozer, M. C., & Wolniewicz, R. em 2003, o Wilcoxon-Mann Whitney U statistic. É utilizada esta aproximação pelo motivo da função AUC não ser diferenciável.

Além disso será utilizado um peso diferente para cada uma das classes na computação da classe, fazendo com que a classe majoritária seja mais penalizada.

In [7]:
print("Quantidade de amostras positivas Up-Selling: %d" % tt_upselling.sum())
print("Quantidade de amostras positivas Churn: %d" % tt_churn.sum())
print("Quantidade de amostras positivas Appentency: %d" % tt_appetency.sum())

Quantidade de amostras positivas Up-Selling: 3682
Quantidade de amostras positivas Churn: 3672
Quantidade de amostras positivas Appentency: 890


# 2) Modelos de IA

## 2.1) Escolha dos modelos

Então, pesquisou-se na literatura sobre trabalhos de modelos não-lineares que foram utilizados para objetivos semelhantes e que obtiveram resultados significativos.

Links de trabalhos base:

https://www.sciencedirect.com/science/article/pii/S1569190X15000386

https://link.springer.com/chapter/10.1007/11875581_56

A partir da análise das características dos dados, do tamanho do dataset e dos resultados anteriores, foram escolhidos 2 modelos para serem treinados e o melhor para cada problema será selecionado. Algumas considerações sobre a escolha dos modelos:

1) Devido à facilidade do modelo em se adaptar às características presentes no dataset deste exercício e devido à área de Deep Learning ser conhecida por obter melhores resultados para diferentes tipos de problemas, o primeiro modelo escolhido foi o de uma Rede Neural Artificial.

2) Devido à facilidade de implementação e custo computacional, sem abrir mão de resultados bastante positivos, o outro modelo selecionado foi o de uma árvore de decisão.

3) O modelo de SVM não foi utilizado, pois a quantidade de dados do dataset é grande e o modelo não é escalável, por conta de seu custo computacional para tratamento de casos não-lineares.

### 2.1.1) Rede Neural Artificial

O primeiro modelo é uma Rede Neural Artificial. Este modelo ficou muito popular nos últimos anos por sua capacidade de encontrar ótimos resultados em vários tipos de problemas de machine learning. Ele tenta imitar o funcionamento dos neurônios humanos, os quais recebem entradas e processam a saída através de impulsos elétricos.No caso do modelo de RNA, este realiza o cálculo da saída através de modelos matemáticos, recebendo os dados de característica como entrada.

O tipo de rede utilizada neste exercício é uma MLP (perceptron multicamadas). Esta rede é do tipo feed-forward - tipo este no qual as conexões entre as camadas da rede não possui ciclos - e possui 3 camadas. A arquitetura de uma rede MLP genérica pode ser vista na figura abaixo, que como pode ser observado, possui uma camada de entrada (cada círculo representa uma característica), uma camada escondida (cada círculo representa um neurônio da camada) e uma camada de saída na qual têm-se a probabilidade de cada uma das classes, no caso de problemas de classificação.

<img src="https://maviccprp.github.io/assets/mlp_files/mlp_graph_latex_full.png" width=400px>

O treinamento das redes neurais utilizam do algoritmo conhecido como $backpropagation$, que utiliza o erro do modelo atual para atualizar os parâmetros (pesos de cada ligação) e obter um melhor modelo.

A rede utilizada utiliza a função de ativação relu ( $f(x) = max(0,x)$ ) para a camada escondida e a função sigmoid ( $f(x) = \frac{1}{1+e^{-x}} $ ) para a camada de saída.

### 2.1.1) Árvore de decisão

Arvóre de decisão é uma técnica de machine learning que consiste em nós de decisão e nós folha. Cada nó de decisão representa um teste X sobre uma característica da entrada e possui ramificações para cada um destes testes, que representam a separação dos dados de acordo com o resultado do teste. Cada nó folha representa uma classe que é resultado da decisão de cada caso.

A figura abaixo mostra, graficamente, um exemplo de estrutura de uma árvore de decisão.

<img src="https://cdn-images-1.medium.com/max/1200/1*EFCePNEkqoGmxm5qR-nqrA.gif" width=400px>


O treinamento de uma árvore de decisão é realizado através de divisão binária recursiva. Consiste em um algoritmo guloso que, em cada passo do algoritmo, são con- sideradas todas as características em cada ponto de divisão da árvore. Cada uma das possibilidades é testada utilizando uma função de custo, e a divisão que apresenta menor custo para a árvore final é selecionada. Estes testes e decisões são repetidos até que seja alcançada a condição de parada, que no caso deste exercício é a profundidade máxima da árvore, que varia entre 3 e 20.

Uma função de custo utilizada em classificadores é o índice de Gini ($Gini = \sum^{k=1}_{k=0}P^{2}_{k}$), sendo que Pk é a fração de classificações corretas para cada uma das classes do problema. O índice Gini mede o quão boa é uma divisão em relação ao quanto as classes estão misturadas nos grupos criados pela divisão.

Para tratamento de oversampling nos modelos de Árvore de decisão serão utilizados apenas o método de oversampling.

## 2.2) Treinamento

Para realizar o treinamento dos modelos propostos, foi utilizada a técnica de validação cruzada k-fold. Esta técnica visa eliminar o viés do conjunto de treinamento e é bastante utilizada no treinamento de algoritmos de machine learning. Os passos para implementação são:

1. permutar as amostras de forma aleatória, em busca de retirar qualquer viés dos dados em relação a distribuição dos mesmos
2. dividir a base de dados em k grupos de tamanhos iguais, ou próximos 
3. para cada um dos k grupos:

    1. separar o grupo para validação
    2. treinar o modelo com os grupos restantes
    3. validar o modelo com o grupo de validação e armazenar o resultado d) descartar o modelo obtido
    
4. resumir o resultado e obter a performance do modelo a partir da média dos resultados

A figura abaixo mostra, graficamente, como o k-fold funciona.

<img src="https://cdn-images-1.medium.com/max/1600/1*me-aJdjnt3ivwAurYkB7PA.png" width=400px>

Outro método utilizado para o treinamento é o de regularização L1. A regularização busca evitar o overfitting do modelo, isto é, ter resultados bons para o conjunto de treinamento, porém não generaliza para o conjunto de teste. A regularização penaliza valores de pesos muito altos, adicionando o somatório do módulo de cada peso na função de loss.

In [36]:
import warnings
warnings.filterwarnings('ignore')
from models_Thullio import NeuralNetwork as nn
from models_Thullio import DecisionTree as dt

### 2.2.1) RNA

In [41]:
import warnings
warnings.filterwarnings('ignore')
#Training NN to Churn
churn_nn = nn(30, 100)
churn_nn.train(df.values, tt_churn['churn'].values)

Valor de AUC para cada um dos conjuntos de validação dos 10 folds:
AUC: 0.62313
AUC: 0.62620
AUC: 0.62027
AUC: 0.57498
AUC: 0.58948
AUC: 0.55556
AUC: 0.61939
AUC: 0.62108
AUC: 0.62580
AUC: 0.62635


In [42]:
import warnings
warnings.filterwarnings('ignore')
#Training NN to appetency
appetency_nn = nn(30, 100)
appetency_nn.train(df.values, tt_appetency['appetency'].values)

Valor de AUC para cada um dos conjuntos de validação dos 10 folds:
AUC: 0.81796
AUC: 0.81398
AUC: 0.81370
AUC: 0.81373
AUC: 0.78928
AUC: 0.81341
AUC: 0.81234
AUC: 0.81600
AUC: 0.78584
AUC: 0.81464


In [43]:
import warnings
warnings.filterwarnings('ignore')
#Training NN to upselling
upselling_nn = nn(30, 100)
upselling_nn.train(df.values, tt_upselling['upselling'].values)

Valor de AUC para cada um dos conjuntos de validação dos 10 folds:
AUC: 0.60585
AUC: 0.60755
AUC: 0.60716
AUC: 0.60578
AUC: 0.60542
AUC: 0.60608
AUC: 0.60611
AUC: 0.60378
AUC: 0.59855
AUC: 0.59810


### 2.2.2) Árvore de Decisão

In [44]:
import warnings
warnings.filterwarnings('ignore')
#Training DT to Churn
churn_dt = dt()
churn_dt.train(df.values, tt_churn['churn'].values)

Melhores parametros:  {'max_depth': 16}
AUC treino:  [0.78481696 0.8062655  0.8243479  0.85936272 0.88354415 0.89899437
 0.9098961  0.92322257 0.93081218 0.93816075 0.94415544 0.95072095
 0.95681637 0.96242733 0.9677217  0.97244287 0.97637138]
AUC validacao:  [0.78479537 0.8059273  0.82373511 0.85803402 0.88115179 0.89599162
 0.90477681 0.91576369 0.92067432 0.92322138 0.92447332 0.92590874
 0.92637282 0.92691245 0.92661026 0.92637282 0.92593032]


In [45]:
import warnings
warnings.filterwarnings('ignore')
#Training DT to Churn
appetency_dt = dt()
appetency_dt.train(df.values, tt_appetency['appetency'].values)

Melhores parametros:  {'max_depth': 19}
AUC treino:  [0.87663409 0.88946696 0.90820154 0.91810788 0.93460599 0.94186068
 0.95257359 0.95870721 0.9663601  0.97240435 0.97941809 0.98403244
 0.98781081 0.9907283  0.9929478  0.99467861 0.99598294]
AUC validacao:  [0.87661372 0.88900428 0.90789045 0.91751171 0.93299735 0.93938098
 0.95007127 0.95336999 0.96047648 0.96390755 0.9690898  0.97021991
 0.97230707 0.97300957 0.97385461 0.97406842 0.97412951]


In [46]:
import warnings
warnings.filterwarnings('ignore')
#Training DT to Churn
upselling_dt = dt()
upselling_dt.train(df.values, tt_upselling['upselling'].values)

Melhores parametros:  {'max_depth': 17}
AUC treino:  [0.78362624 0.80960966 0.84497508 0.86410971 0.88298406 0.89765918
 0.91091657 0.92558688 0.93427921 0.94512213 0.95193614 0.95960894
 0.96530626 0.97059818 0.97531917 0.9790926  0.98279047]
AUC validacao:  [0.78362624 0.80953409 0.84444492 0.86314176 0.88044605 0.89429595
 0.90537156 0.91794767 0.92226564 0.92964938 0.93257481 0.93651496
 0.93713027 0.9381342  0.93827454 0.93806943 0.93781035]


## 3) Teste dos modelos

<font color=blue>_**NOTA**: Como não foram encontrados arquivos com as tabelas de saída para os datasets de teste, tomou-se como premissa que seriam as mesmas tabelas do conjunto de treinamento, por terem o mesmo tamanho (em competições a tabela verdade de saída, geralmente, não podem ser disponibilizada. Então, no caso do exercício, isto foi considerado para gerar algum resultado final). Para uma outra abordagem, os seguintes passos que devem ser feitos:_

- Ignorar o arquivo de teste
- Realizar o EDA
- Separar 10% do conjunto de treinamento para realizar os testes finais, com a parte do treinamento dos modelos não tendo acesso a estes dados
- Testar os modelos nos 10% de dados restantes, verificando assim a não existência de overfitting e garantindo a generalização do modelo.
</font>

Nos testes dos modelos foi obtido um AUC médio de 0.5, o que não reflete os valores obtidos no conjunto de treinamento de validação. Isto representa que houve overfitting dos dados, isto é, o modelo não obteve generalização suficiente para ser bom fora dos dados de treinamento.

In [47]:
print("AUC RNA - Churn: %.5f" % churn_nn.test_auc(df_test.values, tt_churn['churn'].values))
print("AUC Árvore de decisão - Churn: %.5f" % churn_dt.test_auc(df_test.values, tt_churn['churn'].values))

AUC RNA - Churn: 0.50899
AUC Árvore de decisão - Churn: 0.49993


In [48]:
print("AUC RNA - Appetency: %.5f" % appetency_nn.test_auc(df_test.values, tt_appetency['appetency'].values))
print("AUC Árvore de decisão - Appetency: %.5f" % appetency_dt.test_auc(df_test.values, tt_appetency['appetency'].values))

AUC RNA - Appetency: 0.49691
AUC Árvore de decisão - Appetency: 0.50064


In [49]:
print("AUC RNA - Up-Selling: %.5f" % upselling_nn.test_auc(df_test.values, tt_upselling['upselling'].values))
print("AUC Árvore de decisão - Up-Selling: %.5f" % upselling_dt.test_auc(df_test.values, tt_upselling['upselling'].values))

AUC RNA - Up-Selling: 0.50370
AUC Árvore de decisão - Up-Selling: 0.50098


Para resolver o problema identificado, algumas estratégias podem ser levadas em consideração para obter bons modelos:

- Testar outras formas de combater o overfitting
- Testar outras técnicas para tratar o desbalanceamento dos dados
- Testar outras formas de realizar o EDA e tratamento dos dados
- Testar outros modelos, como floresta aleatória, ou até mesmo um ensamble destes métodos em conjunto com a rede neural
- Testar outras arquiteturas para a rede neural